In [4]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_weights_file_path, get_config
from train1 import get_model,get_ds,greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'altair'

In [6]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
config=get_config()

In [9]:
train_dataloader,val_dataloader, tokenizer_src, tokenizer_tgt=get_ds(config)

Max length of source sentence: 471
Max length of target sentence: 471


In [ ]:
model=get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
#train_dataloader,val_dataloader, tokenizer_src, tokenizer_tgt=get_ds(config)
model_filename=get_weights_file_path(config, config['preload'])
print(f"Preloading Model: {model_filename}")
state=torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

In [ ]:
def load_next_batch():
    batch=next(iter(val_dataloader))
    encoder_input=batch['encoder_input'].to(device)
    encoder_mask=batch['encoder_mask'].to(device)
    decoder_input=batch['decoder_input'].to(device)
    decoder_mask=batch['decoder_mask'].to(device)

    encoder_input_tokens=[vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens=[vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]
    model_out=greedy_decode(model,encoder_input,encoder_mask,vocab_src,vocab_tgt,config['seq_len'],device)
    retun batch,encoder_input,decoder_input_tokens
    